In [53]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
from numpy import clip
from torch.autograd import Variable
import torch
from torch import Tensor
from torch.optim import SGD, Adam
from IPython.display import clear_output
import ipywidgets as widgets
from ipywidgets import interact
from torch.nn.functional import softplus
w = widgets.IntText()
%matplotlib inline
%run utils.py
%load_ext Cython
from numpy.random import permutation
M, N = 40, 140

# file to run model on
filename = 'data/split/JC_week1_hour8_40x140.csv'

def log_sum_exp(value):
    m = torch.max(value)
    sum_exp = torch.sum(torch.exp(value - m))
    return m + torch.log(sum_exp)

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [54]:
%%cython
import numpy as np
cimport numpy as np
from itertools import permutations
from sympy.utilities.iterables import multiset_permutations

cdef int M = 40
cdef int N = 140
DTYPE = int
ctypedef np.int_t DTYPE_t


def random_path(int sx, int sy, int tx, int ty, int L):
    """sample L random paths from sx, sy to tx, ty"""
    cdef int hor_dir, ver_dir, X, Y, XY
    
    hor_dir = (tx - sx) > 0
    ver_dir = (ty - sy) > 0
    
    cdef np.ndarray[DTYPE_t, ndim=3] paths = np.zeros([M, N, L], dtype=DTYPE)
    paths[sx,sy,:] = 1
    
    X = abs(tx - sx) 
    Y = abs(ty - sy) 
    XY = X + Y
    
    cdef np.ndarray[DTYPE_t, ndim=1] path = np.zeros(X+Y, dtype=DTYPE)
    path[:X] = 1
    
    cdef int l, i, x, y

    for l in range(L):
        x, y = sx, sy
        np.random.shuffle(path)
        for i in range(XY):
            if path[i]:
                x += 2 * hor_dir - 1
            else:
                y += 2 * ver_dir - 1
            paths[x,y,l] = 1
    return paths


# def all_paths(int sx, int sy, int tx, int ty):
#     """Give all paths from (sx, sy) to (tx, ty)"""
#     cdef int hor_dir, ver_dir, X, Y, XY
#     X = abs(tx - sx) 
#     Y = abs(ty - sy) 
#     XY = X + Y
    
#     hor_dir = (tx - sx) > 0
#     ver_dir = (ty - sy) > 0
    
#     cdef np.ndarray[DTYPE_t, ndim=1] path = np.zeros(X+Y, dtype=DTYPE)
#     path[:X] = 1
#     possible_paths = list(multiset_permutations(path))
    
#     cdef int L = len(possible_paths)
#     cdef np.ndarray[DTYPE_t, ndim=3] paths = np.zeros([M, N, L], dtype=DTYPE)
#     paths[sx,sy,:] = 1

#     cdef int l, i, x, y

#     for l in range(L):
#         x, y = sx, sy
#         path = np.array(list(possible_paths[l]))
#         for i in range(XY):
#             if path[i]:
#                 x += 2 * hor_dir - 1
#             else:
#                 y += 2 * ver_dir - 1
#             paths[x,y,l] = 1
#     return paths

In [61]:
np.abs(df['ty'] - df['sy']).mean()

15.756249840075741

In [55]:
weights = Variable((10 * torch.ones((M,N))).float(), requires_grad=True)

# hyperparameters of the model
L = 20
s2 = 10 ** 2 # sigma2 = (10 s) ** 2

df = pd.read_csv(filename)

optimizer = Adam([weights], lr = 0.05) # using Adam since we are doing some stochastic optimization things
num_epochs = 5
def loss_fn(paths, t):
    """Implement the loss function"""
    weighted_paths = (Variable(Tensor(paths.astype(float)))
                              * weights.unsqueeze(-1))
    weight_sums = weighted_paths.sum(dim=0).sum(dim=0)
    first = (-1/(2 * L * s2) * (t - weight_sums).pow(2).sum())
    second = - weighted_paths.sum() / L
    third = - log_sum_exp(-weight_sums)
    loss = -(first + second + third)
    return loss 

    

pbar = tqdm(total=num_epochs * len(df))
losses = []

for epoch in range(num_epochs):
    for _, duration, sx, sy, tx, ty in (df.sample(frac=1).itertuples()):
        pbar.update(1)
        paths = random_path(int(sx), int(sy), int(tx), int(ty), L)
        t = float(duration)
        loss = loss_fn(paths, t)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.data[0])

A Jupyter Widget

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/jiafengchen/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/jiafengchen/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/jiafengchen/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



KeyboardInterrupt: 